In [67]:
import numpy as np
import pandas as pd
import numpy as np
import scipy.io
import statsmodels.api as sm

In [68]:
beta = np.loadtxt("betas1.csv", delimiter=",")

In [69]:
mat = scipy.io.loadmat('china.mat')
mret = np.array(mat['china_monthly_ri'])
market = np.array(mat['china_monthly_mv'])
rankrt = np.stack([(mret[:,i] - mret[:,i-1])/mret[:,i-1] for i in range(1,mret.shape[1])], axis = -1)
mk = np.stack([market[:,i] for i in range(1,mret.shape[1])], axis = -1)
momentum = np.stack([np.prod(rankrt[:,i-11:i] + 1,axis = 1) -1 for i in range(11,rankrt.shape[1])], axis = -1)
momentum= np.concatenate((np.zeros([momentum.shape[0],11]), momentum), axis=1)
rf = pd.read_csv('F-F_Research_Data_Factors.CSV')
rf = rf[rf['Time'] >= mat['mdates'][0][0]]['RF']

/Users/ningpeiding/anaconda3/lib/python3.6/site-packages/scipy/io/matlab/mio.py:142: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [70]:
dwse = np.array(mat['china_dwse'])
dwse = np.stack([dwse[:,i] for i in range(1, dwse.shape[1])], axis = -1)

In [71]:
dweb = np.array(mat['china_dweb'])
dweb = np.stack([dweb[:,i] for i in range(1, dweb.shape[1])], axis = -1)

In [109]:
nc = rankrt.shape[1]
nr = rankrt.shape[0]

from sklearn.linear_model import LinearRegression
# bm = dweb/mk
result = []
for i in range(98, nc):
        index_num = (~np.isnan(rankrt[:,i])) & (~np.isnan(dwse[:,i - 7]))& (~np.isnan(mk[:,i - 1])) &(~np.isnan(beta[:,i]))&(~np.isnan(dweb[:,i - 7]))
        bm = dwse[index_num,i - 7]/mk[index_num,i - 1]
        if index_num.sum() <= 0:
            continue
        reg = LinearRegression().fit(np.stack((beta[index_num,i],bm, dweb[index_num,i - 7]), axis=-1), rankrt[index_num,i])
        score = reg.score(np.stack((beta[index_num,i],bm, dweb[index_num,i - 7]), axis=-1), rankrt[index_num,i])
        reg1 = LinearRegression().fit(np.stack((beta[index_num,i], dwse[index_num,i - 7]), axis=-1), rankrt[index_num,i])
        score1 = reg1.score(np.stack((beta[index_num,i], dwse[index_num,i - 7]), axis=-1), rankrt[index_num,i])
        reg2 = LinearRegression().fit(beta[index_num,i].reshape(-1, 1), rankrt[index_num,i])
        score2 = reg2.score(beta[index_num,i].reshape(-1, 1), rankrt[index_num,i])
        result.append([[reg.coef_,score],[reg1.coef_, score1], [reg2.coef_, score2]])
    
    
    

In [110]:
h = np.array(result)

# Three Factors: Beta, Book to Market and EBITDA

In [111]:
h[:,0,0].mean(axis = 0)

array([ 1.43329479e-03,  8.78664790e-06, -1.03121102e-08])

In [112]:
h[:,0,1].mean()

0.030882596967030135

# Two Factors: Beta and EBITDA

In [113]:
h[:,1,0].mean(axis = 0)

array([ 1.83883115e-03, -1.18096210e-09])

In [114]:
h[:,1,1].mean()

0.01861724958834111

# One Factor: Beta

In [115]:
h[:,2,0].mean(axis = 0)

array([0.00116626])

In [116]:
h[:,2,1].mean(axis = 0)

0.009690956161739157